<a href="https://colab.research.google.com/github/ajiteshreddy24/AuthentiText/blob/main/nlpprojectcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
def read_tsv_mrpc(input_file):

    tab = []
    with open(input_file, "r", encoding="utf8") as f:
        next(f)
        for line in f:
            tab.append(line.strip().split('\t'))

    dfObj = pd.DataFrame(tab)

    df = pd.DataFrame({'IDS': [i for i in range(len(tab))],
                       'sentence': dfObj[3],
                       'paraphrase': dfObj[4],
                       'quality': dfObj[0].astype('int64')})

    return df

In [ ]:
df = pd.DataFrame(read_tsv_mrpc("msr_paraphrase_train.txt"))

In [ ]:
df

,IDS,sentence,paraphrase,quality
0,0,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi...",1
1,1,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...,0
2,2,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an...",1
3,3,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ...",0
4,4,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...,1
...,...,...,...,...
4071,4071,"""At this point, Mr. Brando announced: 'Somebod...","Brando said that ""somebody ought to put a bull...",1
4072,4072,"Martin, 58, will be freed today after serving ...",Martin served two thirds of a five-year senten...,0
4073,4073,"""We have concluded that the outlook for price ...","In a statement, the ECB said the outlook for p...",1
4074,4074,The notification was first reported Friday by ...,MSNBC.com first reported the CIA request on Fr...,1


In [ ]:
import pickle
from array import array
import numpy as np
import random
import gensim


index = []
dict = []
not_started = True

#model = gensim.models.Word2Vec.load_word2vec_format('../../../../word2vec/GoogleNews-vectors-small-300.bin', binary=True)
#model = gensim.models.Word2Vec.load_word2vec_format('../../../../word2vec/GoogleNews-vectors-paraphrase-300.bin', binary=True)
#model.init_sims(replace=True)

def clean_sent_cond(sent):
    tok = sent.split(" ")
    if(len(tok) < 13):
        return True
    if max(map(lambda x: len(x), tok)) < 4:
        return True
    return False

def preprocess(model):
    dict_sentences = {}
    reverse_dict = {}
    match_dictionary = {}
    pair_list = []
    import sys
    i = 0
    k = 0
    maxlen = 0
    # this reads in one line at a time from stdin
    for line in sys.stdin:
        i+=1
        tokens = line.split("\t")
        sent1 = tokens[0]
        sent2 = tokens[1]

        if clean_sent_cond(sent1) or clean_sent_cond(sent2):
            continue
        else:
            k += 1

        if not sent1 in dict_sentences:
            dict_sentences[sent1] = len(dict_sentences) + 1
        if not sent2 in dict_sentences:
            dict_sentences[sent2] = len(dict_sentences) + 1
        index_1 = dict_sentences[sent1]
        index_2 = dict_sentences[sent2]

        if not index_1 in match_dictionary:
            match_dictionary[index_1] = []
        if not index_2 in match_dictionary:
            match_dictionary[index_2] = []
        match_dictionary[index_1].append(index_2)
        match_dictionary[index_2].append(index_1)
        pair_list.append((index_1, index_2))

        if i % 10000 == 0:
            print(str(k) + "/" + str(i))
        if k == 500000:
            break;

    i = 0
    for entry in dict_sentences:
        simple_sent1 = filter(lambda x: len(x) > 1, data_helpers.clean_str(entry).split(" "))
        sent1 = data_helpers.build_input_data(data_helpers.pad_sentences([simple_sent1], 40, padding_word="<PAD/>"),
                                          model.vocab)
        reverse_dict[dict_sentences[entry]] = sent1
        if i % 10000 == 0:
            print(i)
        i += 1

    random.shuffle(pair_list)
    pickle.dump(reverse_dict, open("sentences_small_x", "wb"), protocol=pickle.HIGHEST_PROTOCOL)
    print("writing sentences " + str(len(reverse_dict)))
    pickle.dump(match_dictionary, open("pairs_index_small_x", "wb"), protocol=pickle.HIGHEST_PROTOCOL)
    print("writing map " + str(len(match_dictionary)))
    pickle.dump(pair_list, open("pairs_list_small_x", "wb"), protocol=pickle.HIGHEST_PROTOCOL)
    print("pairs " + str(len(pair_list)))


def read_and_index():
    dict = pickle.load(open("sentences_small_x", "rb"))
    print("opened dict")
    index = pickle.load(open("pairs_index_small_x", "rb"))
    print("opened index")
    pairs_list = pickle.load(open("pairs_list_small_x", "rb"))
    print("opened pairs list")

    random.seed(0)
    return index, dict, pairs_list

def print_prop(numbers, model):
    stringA = ""
    for x in numbers:
        if x < len(model.index2word):
            stringA += model.index2word[x].encode('ascii', 'ignore') + " "
    return stringA

def generate_train_batch(pos, batchsize, index, dict, pairs_list):
    batch = []
    for i in range(int(batchsize / 2)):
        index_pair = (pos * int(batchsize / 2) + i) % len(index) + 1#random.randint(1, len(index))

        index_1, index_2 = pairs_list[index_pair]
        index_neg = -1
        while index_neg == -1:
            index_neg = random.randint(1, len(index))
            if index_neg in index[index_1]:
                index_neg = -1
        if len(dict[index_neg][0]) != 40 or len(dict[index_1][0]) != 40 or len(dict[index_2][0]) != 40:
            print("Ouch")
            continue

        if len(batch) == 0:
            batch = np.array([dict[index_1], dict[index_2], [1, 0]])
        else:
            batch = np.insert(batch, 0, np.array([dict[index_1], dict[index_2], [1, 0]]))

        batch = np.insert(batch, 0, np.array([dict[index_1], dict[index_neg], [0, 1]]))
    result1 = []
    result2 = []
    result3 = []
    for i in range(int(len(batch) / 3)):
        result1.append(batch[3 * i][0])
        result2.append(batch[3 * i + 1][0])
        result3.append(batch[3 * i + 2])

    return [tuple(result1), tuple(result2), tuple(result3)]

#preprocess(model)

#dict, index, pairs = read_and_index()

#for i in range(1,20):
#    print str(pairs[i][0]) + ":" + str(pairs[i][1]) + "  " + print_prop(index[pairs[i][0]][0], model)
#    print print_prop(index[pairs[i][1]][0], model)
#    print "\n"

In [ ]:
import nltk
from nltk.corpus import stopwords
from gensim.models import KeyedVectors

# Load stop words
stop_words = stopwords.words('english')

# Function to preprocess text
def preprocess_text(text):
    text = text.lower()  # Lowercase
    text = nltk.word_tokenize(text)  # Tokenize
    text = [word for word in text if word not in stop_words]  # Remove stop words
    return text

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

import re

def preprocess_text(text):
    text = text.lower()
    text = nltk.word_tokenize(text)
    text = [word for word in text if word.isalpha()]  # Keep only words containing alphabetic characters
    text = [word for word in text if len(word) > 1]  # Keep only words with length greater than 1
    text = [word for word in text if word not in stop_words]  # Remove stop words

    # Join the remaining words into a sentence
    text = ' '.join(text)

    # Remove multiple commas and extra spaces
    text = re.sub(r',+', ',', text)
    text = re.sub(r'\s+', ' ', text)

    return text.strip()  # Strip any leading or trailing spaces
df['processed_sentence'] = df['sentence'].apply(preprocess_text)
df['processed_paraphrase'] = df['paraphrase'].apply(preprocess_text)


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
len((df["processed_sentence"][1]))

52

In [ ]:
df["processed_sentence"][1]


'yucaipa owned dominick selling chain safeway billion'

In [ ]:
df

,IDS,sentence,paraphrase,quality,processed_sentence,processed_paraphrase
0,0,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi...",1,amrozi accused brother called witness delibera...,referring witness amrozi accused brother delib...
1,1,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...,0,yucaipa owned dominick selling chain safeway b...,yucaipa bought dominick million sold safeway b...
2,2,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an...",1,published advertisement internet june offering...,june ship owners published advertisement inter...
3,3,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ...",0,around gmt tab shares cents earlier set record...,tab shares jumped cents set record closing high
4,4,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...,1,stock rose percent close friday new york stock...,pg shares jumped percent new york stock exchan...
...,...,...,...,...,...,...
4071,4071,"""At this point, Mr. Brando announced: 'Somebod...","Brando said that ""somebody ought to put a bull...",1,point brando announced ought put bullet head m...,brando said somebody ought put bullet head acc...
4072,4072,"Martin, 58, will be freed today after serving ...",Martin served two thirds of a five-year senten...,0,martin freed today serving two thirds sentence...,martin served two thirds sentence manslaughter...
4073,4073,"""We have concluded that the outlook for price ...","In a statement, the ECB said the outlook for p...",1,concluded outlook price stability medium term ...,statement ecb said outlook price stability med...
4074,4074,The notification was first reported Friday by ...,MSNBC.com first reported the CIA request on Fr...,1,notification first reported friday msnbc,first reported cia request friday


In [ ]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import pandas as pd

# Download NLTK resources (needed for lemmatization)


# Initialize WordNet lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to map NLTK's POS tags to WordNet's POS tags
def get_wordnet_pos(tag):
    if tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('R'):
        return wordnet.ADV
    elif tag.startswith('J'):
        return wordnet.ADJ
    else:
        return None

# Function to perform lemmatization on a single word
def lemmatize_word(word, pos_tag):
    wn_pos = get_wordnet_pos(pos_tag)
    if wn_pos:
        return lemmatizer.lemmatize(word, pos=wn_pos)
    else:
        return word

# Function to lemmatize a sentence
def lemmatize_sentence(sentence):
    # Tokenize the sentence into words
    words = nltk.word_tokenize(sentence)

    # Get part-of-speech tags for words
    pos_tags = nltk.pos_tag(words)

    # Lemmatize each word in the sentence
    lemmatized_words = [lemmatize_word(word, tag) for word, tag in pos_tags]

    # Join the lemmatized words back into a sentence
    lemmatized_sentence = ' '.join(lemmatized_words)
    return lemmatized_sentence



# Apply lemmatization to each column
df['lemmatized_sentence'] = df['processed_sentence'].apply(lemmatize_sentence)
df['lemmatized_paraphrase'] = df['processed_paraphrase'].apply(lemmatize_sentence)

# Display the DataFrame with lemmatized columns
print(df)


       IDS                                           sentence  \
0        0  Amrozi accused his brother, whom he called "th...   
1        1  Yucaipa owned Dominick's before selling the ch...   
2        2  They had published an advertisement on the Int...   
3        3  Around 0335 GMT, Tab shares were up 19 cents, ...   
4        4  The stock rose $2.11, or about 11 percent, to ...   
...    ...                                                ...   
4071  4071  "At this point, Mr. Brando announced: 'Somebod...   
4072  4072  Martin, 58, will be freed today after serving ...   
4073  4073  "We have concluded that the outlook for price ...   
4074  4074  The notification was first reported Friday by ...   
4075  4075  The 30-year bond US30YT=RR rose 22/32 for a yi...   

                                             paraphrase  quality  \
0     Referring to him as only "the witness", Amrozi...        1   
1     Yucaipa bought Dominick's in 1995 for $693 mil...        0   
2     On June 1

In [ ]:
df

,IDS,sentence,paraphrase,quality,processed_sentence,processed_paraphrase,lemmatized_sentence,lemmatized_paraphrase
0,0,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi...",1,amrozi accused brother called witness delibera...,referring witness amrozi accused brother delib...,amrozi accuse brother call witness deliberatel...,refer witness amrozi accuse brother deliberate...
1,1,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...,0,yucaipa owned dominick selling chain safeway b...,yucaipa bought dominick million sold safeway b...,yucaipa own dominick selling chain safeway bil...,yucaipa buy dominick million sell safeway billion
2,2,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an...",1,published advertisement internet june offering...,june ship owners published advertisement inter...,publish advertisement internet june offering c...,june ship owner publish advertisement internet...
3,3,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ...",0,around gmt tab shares cents earlier set record...,tab shares jumped cents set record closing high,around gmt tab share cent earlier set record high,tab share jump cent set record closing high
4,4,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...,1,stock rose percent close friday new york stock...,pg shares jumped percent new york stock exchan...,stock rise percent close friday new york stock...,pg share jump percent new york stock exchange ...
...,...,...,...,...,...,...,...,...
4071,4071,"""At this point, Mr. Brando announced: 'Somebod...","Brando said that ""somebody ought to put a bull...",1,point brando announced ought put bullet head m...,brando said somebody ought put bullet head acc...,point brando announce ought put bullet head mo...,brando say somebody ought put bullet head acco...
4072,4072,"Martin, 58, will be freed today after serving ...",Martin served two thirds of a five-year senten...,0,martin freed today serving two thirds sentence...,martin served two thirds sentence manslaughter...,martin free today serve two third sentence man...,martin serve two third sentence manslaughter b...
4073,4073,"""We have concluded that the outlook for price ...","In a statement, the ECB said the outlook for p...",1,concluded outlook price stability medium term ...,statement ecb said outlook price stability med...,concluded outlook price stability medium term ...,statement ecb say outlook price stability medi...
4074,4074,The notification was first reported Friday by ...,MSNBC.com first reported the CIA request on Fr...,1,notification first reported friday msnbc,first reported cia request friday,notification first report friday msnbc,first report cia request friday


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [ ]:
df

,IDS,sentence,paraphrase,quality,processed_sentence,processed_paraphrase,lemmatized_sentence,lemmatized_paraphrase
0,0,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi...",1,amrozi accused brother called witness delibera...,referring witness amrozi accused brother delib...,amrozi accuse brother call witness deliberatel...,refer witness amrozi accuse brother deliberate...
1,1,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...,0,yucaipa owned dominick selling chain safeway b...,yucaipa bought dominick million sold safeway b...,yucaipa own dominick selling chain safeway bil...,yucaipa buy dominick million sell safeway billion
2,2,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an...",1,published advertisement internet june offering...,june ship owners published advertisement inter...,publish advertisement internet june offering c...,june ship owner publish advertisement internet...
3,3,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ...",0,around gmt tab shares cents earlier set record...,tab shares jumped cents set record closing high,around gmt tab share cent earlier set record high,tab share jump cent set record closing high
4,4,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...,1,stock rose percent close friday new york stock...,pg shares jumped percent new york stock exchan...,stock rise percent close friday new york stock...,pg share jump percent new york stock exchange ...
...,...,...,...,...,...,...,...,...
4071,4071,"""At this point, Mr. Brando announced: 'Somebod...","Brando said that ""somebody ought to put a bull...",1,point brando announced ought put bullet head m...,brando said somebody ought put bullet head acc...,point brando announce ought put bullet head mo...,brando say somebody ought put bullet head acco...
4072,4072,"Martin, 58, will be freed today after serving ...",Martin served two thirds of a five-year senten...,0,martin freed today serving two thirds sentence...,martin served two thirds sentence manslaughter...,martin free today serve two third sentence man...,martin serve two third sentence manslaughter b...
4073,4073,"""We have concluded that the outlook for price ...","In a statement, the ECB said the outlook for p...",1,concluded outlook price stability medium term ...,statement ecb said outlook price stability med...,concluded outlook price stability medium term ...,statement ecb say outlook price stability medi...
4074,4074,The notification was first reported Friday by ...,MSNBC.com first reported the CIA request on Fr...,1,notification first reported friday msnbc,first reported cia request friday,notification first report friday msnbc,first report cia request friday


In [ ]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Assuming nltk's data is downloaded; if not, run:
# nltk.download('punkt')
# nltk.download('stopwords')

# Dummy DataFrame creation for demonstration (since 'df' is not defined in the provided code)
# Example format of df with some text data:

# Tokenize sentences in the DataFrame
df['tokenized_sentence'] = df['lemmatized_sentence'].apply(word_tokenize)
df['tokenized_paraphrase'] = df['lemmatized_paraphrase'].apply(word_tokenize)

# Function to train Word2Vec model on the preprocessed sentences
def train_word2vec_model(sentences):
    word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
    return word2vec_model

# Train Word2Vec model on tokenized sentences
word2vec_model_sentence = train_word2vec_model(df["tokenized_sentence"].tolist())
word2vec_model_paraphrase = train_word2vec_model(df["tokenized_paraphrase"].tolist())

# Function to convert tokenized sentences to vectors using Word2Vec model
def sentence_to_vectors(tokens, word2vec_model):
    vectors = [word2vec_model.wv[word] for word in tokens if word in word2vec_model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)  # Return zero vector if no words are found in the model

# Convert preprocessed sentences to vectors
df['sentence_vector'] = df['tokenized_sentence'].apply(lambda x: sentence_to_vectors(x, word2vec_model_sentence))
df['paraphrase_vector'] = df['tokenized_paraphrase'].apply(lambda x: sentence_to_vectors(x, word2vec_model_paraphrase))

# Display the DataFrame with vector representations
print(df[['sentence_vector', 'paraphrase_vector']])


                                        sentence_vector  \
0     [-0.009033697, 0.020560462, 0.010047546, -0.00...   
1     [-0.002838067, 0.013864668, 0.0074791783, -0.0...   
2     [-0.00996386, 0.0302961, 0.019268876, 0.003712...   
3     [-0.016323583, 0.047898483, 0.034866557, 0.005...   
4     [-0.038968604, 0.09126884, 0.053458165, 0.0002...   
...                                                 ...   
4071  [-0.010764404, 0.029936863, 0.017743299, -0.00...   
4072  [-0.010916585, 0.027640522, 0.017130597, 0.000...   
4073  [-0.0121092, 0.047071077, 0.02728192, 0.006032...   
4074  [-0.022425227, 0.06336243, 0.039433856, 0.0018...   
4075  [-0.029148439, 0.072722524, 0.047230203, 0.005...   

                                      paraphrase_vector  
0     [-0.0072319414, 0.0132438075, 0.0026644734, 0....  
1     [-0.02956066, 0.04934479, 0.02019783, 0.007735...  
2     [-0.018806985, 0.030951645, 0.0060837115, 0.00...  
3     [-0.032473966, 0.0616716, 0.018584171, 0.01479...  
4

In [ ]:
len(df["sentence_vector"][1])

100

In [ ]:
len(df["lemmatized_sentence"][1])

50

In [ ]:
df

,IDS,sentence,paraphrase,quality,processed_sentence,processed_paraphrase,lemmatized_sentence,lemmatized_paraphrase,tokenized_sentence,tokenized_paraphrase,sentence_vector,paraphrase_vector
0,0,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi...",1,amrozi accused brother called witness delibera...,referring witness amrozi accused brother delib...,amrozi accuse brother call witness deliberatel...,refer witness amrozi accuse brother deliberate...,"[amrozi, accuse, brother, call, witness, delib...","[refer, witness, amrozi, accuse, brother, deli...","[-0.009033697, 0.020560462, 0.010047546, -0.00...","[-0.0072319414, 0.0132438075, 0.0026644734, 0...."
1,1,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...,0,yucaipa owned dominick selling chain safeway b...,yucaipa bought dominick million sold safeway b...,yucaipa own dominick selling chain safeway bil...,yucaipa buy dominick million sell safeway billion,"[yucaipa, own, dominick, selling, chain, safew...","[yucaipa, buy, dominick, million, sell, safewa...","[-0.002838067, 0.013864668, 0.0074791783, -0.0...","[-0.02956066, 0.04934479, 0.02019783, 0.007735..."
2,2,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an...",1,published advertisement internet june offering...,june ship owners published advertisement inter...,publish advertisement internet june offering c...,june ship owner publish advertisement internet...,"[publish, advertisement, internet, june, offer...","[june, ship, owner, publish, advertisement, in...","[-0.00996386, 0.0302961, 0.019268876, 0.003712...","[-0.018806985, 0.030951645, 0.0060837115, 0.00..."
3,3,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ...",0,around gmt tab shares cents earlier set record...,tab shares jumped cents set record closing high,around gmt tab share cent earlier set record high,tab share jump cent set record closing high,"[around, gmt, tab, share, cent, earlier, set, ...","[tab, share, jump, cent, set, record, closing,...","[-0.016323583, 0.047898483, 0.034866557, 0.005...","[-0.032473966, 0.0616716, 0.018584171, 0.01479..."
4,4,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...,1,stock rose percent close friday new york stock...,pg shares jumped percent new york stock exchan...,stock rise percent close friday new york stock...,pg share jump percent new york stock exchange ...,"[stock, rise, percent, close, friday, new, yor...","[pg, share, jump, percent, new, york, stock, e...","[-0.038968604, 0.09126884, 0.053458165, 0.0002...","[-0.05403412, 0.10484073, 0.027676746, 0.02440..."
...,...,...,...,...,...,...,...,...,...,...,...,...
4071,4071,"""At this point, Mr. Brando announced: 'Somebod...","Brando said that ""somebody ought to put a bull...",1,point brando announced ought put bullet head m...,brando said somebody ought put bullet head acc...,point brando announce ought put bullet head mo...,brando say somebody ought put bullet head acco...,"[point, brando, announce, ought, put, bullet, ...","[brando, say, somebody, ought, put, bullet, he...","[-0.010764404, 0.029936863, 0.017743299, -0.00...","[-0.035072923, 0.05492689, 0.018419351, 0.0170..."
4072,4072,"Martin, 58, will be freed today after serving ...",Martin served two thirds of a five-year senten...,0,martin freed today serving two thirds sentence...,martin served two thirds sentence manslaughter...,martin free today serve two third sentence man...,martin serve two third sentence manslaughter b...,"[martin, free, today, serve, two, third, sente...","[martin, serve, two, third, sentence, manslaug...","[-0.010916585, 0.027640522, 0.017130597, 0.000...","[-0.020992694, 0.032085575, 0.012728949, 0.010..."
4073,4073,"""We have concluded that the outlook for price ...","In a statement, the ECB said the outlook for p...",1,concluded outlook

In [ ]:
df["quality"].value_counts()

quality
1    2753
0    1323
Name: count, dtype: int64

In [ ]:
len(df)

4076

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, Input
import tensorflow.keras.backend as K

# Define Siamese network architecture with more complex LSTM layers
def siamese_network(input_shape):
    input_sentence = Input(shape=input_shape)
    input_paraphrase = Input(shape=input_shape)

    # Shared LSTM layers with increased complexity
    lstm = layers.LSTM(128, return_sequences=True)(input_sentence)
    lstm = layers.LSTM(64, return_sequences=True)(lstm)
    lstm = layers.LSTM(32)(lstm)
    sentence_branch = lstm

    lstm = layers.LSTM(128, return_sequences=True)(input_paraphrase)
    lstm = layers.LSTM(64, return_sequences=True)(lstm)
    lstm = layers.LSTM(32)(lstm)
    paraphrase_branch = lstm

    # Shared dense layer
    shared_dense = layers.Dense(64, activation='relu')
    sentence_output = shared_dense(sentence_branch)
    paraphrase_output = shared_dense(paraphrase_branch)

    return models.Model(inputs=[input_sentence, input_paraphrase], outputs=[sentence_output, paraphrase_output])

# Contrastive loss function
def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

# Load and preprocess data


# Convert to numpy arrays
X_sentence = df['sentence_vector'].to_numpy()
X_paraphrase = df['paraphrase_vector'].to_numpy()
y = df['quality'].astype(float).to_numpy()

# Reshape sentence and paraphrase vectors
X_sentence = np.stack(X_sentence)  # Stack into a 2D array
X_paraphrase = np.stack(X_paraphrase)  # Stack into a 2D array

# Split data into train and test sets
X_train_sentence, X_test_sentence, X_train_paraphrase, X_test_paraphrase, y_train, y_test = train_test_split(X_sentence, X_paraphrase, y, test_size=0.2, random_state=101)

# Reshape sentence and paraphrase vectors for LSTM input
input_shape = (X_train_sentence.shape[1], 1)
X_train_sentence = X_train_sentence[:, :, np.newaxis]
X_test_sentence = X_test_sentence[:, :, np.newaxis]
X_train_paraphrase = X_train_paraphrase[:, :, np.newaxis]
X_test_paraphrase = X_test_paraphrase[:, :, np.newaxis]

# Build Siamese network with increased complexity
siamese = siamese_network(input_shape=input_shape)

# Compile the model without early stopping
siamese.compile(optimizer='adam', loss=contrastive_loss, metrics=['accuracy'])

# Train the model without early stopping
history = siamese.fit([X_train_sentence, X_train_paraphrase], [y_train, y_train], batch_size=32, epochs=10,
                      validation_data=([X_test_sentence, X_test_paraphrase], [y_test, y_test]), verbose=1)

# Evaluate the model on test set
test_results = siamese.evaluate([X_test_sentence, X_test_paraphrase], [y_test, y_test], verbose=0)
test_loss = test_results[0]

# Calculate accuracy with a threshold of 0.5
sentence_test_accuracy = np.mean(np.round(test_results[3]) == y_test)
paraphrase_test_accuracy = np.mean(np.round(test_results[4]) == y_test)

print(f'Test Loss: {test_loss:.4f}')
print(f'Sentence Accuracy: {sentence_test_accuracy:.4f}')
print(f'Paraphrase Accuracy: {paraphrase_test_accuracy:.4f}')


Epoch 1/10
102/102 [==============================] - 60s 471ms/step - loss: 0.5061 - dense_loss: 0.2545 - dense_1_loss: 0.2516 - dense_accuracy: 0.0000e+00 - dense_1_accuracy: 0.0000e+00 - val_loss: 0.5176 - val_dense_loss: 0.2589 - val_dense_1_loss: 0.2587 - val_dense_accuracy: 0.0000e+00 - val_dense_1_accuracy: 0.0000e+00
Epoch 2/10
102/102 [==============================] - 46s 450ms/step - loss: 0.4860 - dense_loss: 0.2430 - dense_1_loss: 0.2430 - dense_accuracy: 0.0107 - dense_1_accuracy: 0.0000e+00 - val_loss: 0.5220 - val_dense_loss: 0.2608 - val_dense_1_loss: 0.2612 - val_dense_accuracy: 0.0000e+00 - val_dense_1_accuracy: 0.0000e+00
Epoch 3/10
102/102 [==============================] - 52s 512ms/step - loss: 0.4854 - dense_loss: 0.2427 - dense_1_loss: 0.2427 - dense_accuracy: 0.0184 - dense_1_accuracy: 0.0000e+00 - val_loss: 0.5193 - val_dense_loss: 0.2597 - val_dense_1_loss: 0.2596 - val_dense_accuracy: 0.0000e+00 - val_dense_1_accuracy: 0.0000e+00
Epoch 4/10
102/102 [=======

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, Input

# Load and preprocess data


# Convert to numpy arrays
X_sentence = df['sentence_vector'].to_numpy()
X_paraphrase = df['paraphrase_vector'].to_numpy()
y = df['quality'].astype(float).to_numpy()

# Reshape sentence and paraphrase vectors
X_sentence = np.stack(X_sentence)  # Stack into a 2D array
X_paraphrase = np.stack(X_paraphrase)  # Stack into a 2D array

# Split data into train and test sets
X_train_sentence, X_test_sentence, X_train_paraphrase, X_test_paraphrase, y_train, y_test = train_test_split(X_sentence, X_paraphrase, y, test_size=0.2, random_state=101)

# Reshape sentence and paraphrase vectors for LSTM input
input_shape = (X_train_sentence.shape[1], 1)
X_train_sentence = X_train_sentence[:, :, np.newaxis]
X_test_sentence = X_test_sentence[:, :, np.newaxis]
X_train_paraphrase = X_train_paraphrase[:, :, np.newaxis]
X_test_paraphrase = X_test_paraphrase[:, :, np.newaxis]

# Define Siamese network architecture with increased complexity and batch normalization
def siamese_network(input_shape):
    input_sentence = Input(shape=input_shape)
    input_paraphrase = Input(shape=input_shape)

    # Shared LSTM layers with increased complexity and batch normalization
    lstm = layers.LSTM(256, return_sequences=True)(input_sentence)
    lstm = layers.BatchNormalization()(lstm)
    lstm = layers.LSTM(128, return_sequences=True)(lstm)
    lstm = layers.BatchNormalization()(lstm)
    lstm = layers.LSTM(64)(lstm)
    sentence_branch = lstm

    lstm = layers.LSTM(256, return_sequences=True)(input_paraphrase)
    lstm = layers.BatchNormalization()(lstm)
    lstm = layers.LSTM(128, return_sequences=True)(lstm)
    lstm = layers.BatchNormalization()(lstm)
    lstm = layers.LSTM(64)(lstm)
    paraphrase_branch = lstm

    # Shared dense layers with dropout
    shared_dense = layers.Dense(128, activation='relu')
    sentence_output = shared_dense(sentence_branch)
    sentence_output = layers.Dropout(0.5)(sentence_output)  # Dropout regularization
    paraphrase_output = shared_dense(paraphrase_branch)
    paraphrase_output = layers.Dropout(0.5)(paraphrase_output)  # Dropout regularization

    return models.Model(inputs=[input_sentence, input_paraphrase], outputs=[sentence_output, paraphrase_output])

# Build Siamese network
siamese = siamese_network(input_shape=input_shape)

# Compile the model with adjusted learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)  # Adjust learning rate
siamese.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = siamese.fit([X_train_sentence, X_train_paraphrase], [y_train, y_train], batch_size=32, epochs=20,
                      validation_data=([X_test_sentence, X_test_paraphrase], [y_test, y_test]), verbose=1)

# Evaluate the model on test set
test_results = siamese.evaluate([X_test_sentence, X_test_paraphrase], [y_test, y_test], verbose=0)
test_loss = test_results[0]

# Calculate accuracy with a threshold of 0.5
sentence_test_accuracy = np.mean(np.round(test_results[3]) == y_test)
paraphrase_test_accuracy = np.mean(np.round(test_results[4]) == y_test)

print(f'Test Loss: {test_loss:.4f}')
print(f'Sentence Accuracy: {sentence_test_accuracy:.4f}')
print(f'Paraphrase Accuracy: {paraphrase_test_accuracy:.4f}')


Epoch 1/20
102/102 [==============================] - 149s 1s/step - loss: 15.1465 - dropout_loss: 7.6494 - dropout_1_loss: 7.4971 - dropout_accuracy: 0.0000e+00 - dropout_1_accuracy: 0.0144 - val_loss: 10.1947 - val_dropout_loss: 5.3170 - val_dropout_1_loss: 4.8777 - val_dropout_accuracy: 0.0000e+00 - val_dropout_1_accuracy: 0.0000e+00
Epoch 2/20
102/102 [==============================] - 125s 1s/step - loss: 13.9869 - dropout_loss: 6.9200 - dropout_1_loss: 7.0669 - dropout_accuracy: 0.0000e+00 - dropout_1_accuracy: 0.0000e+00 - val_loss: 9.3894 - val_dropout_loss: 4.5444 - val_dropout_1_loss: 4.8450 - val_dropout_accuracy: 0.0000e+00 - val_dropout_1_accuracy: 0.0000e+00
Epoch 3/20
102/102 [==============================] - 128s 1s/step - loss: 13.7316 - dropout_loss: 6.7218 - dropout_1_loss: 7.0098 - dropout_accuracy: 6.1350e-04 - dropout_1_accuracy: 9.2025e-04 - val_loss: 8.3311 - val_dropout_loss: 3.9172 - val_dropout_1_loss: 4.4138 - val_dropout_accuracy: 0.0000e+00 - val_dropout_